# Post-processing example

In [ ]:
#for more advanced hisfile operations: https://github.com/Deltares/dfm_tools/blob/main/tests/examples/postprocess_hisnc.py
#for more advanced mapfile operations: https://github.com/Deltares/dfm_tools/blob/main/tests/examples/postprocess_mapnc_ugrid.py
# -*- coding: utf-8 -*-
"""
author: Lorinc Meszaros, Deltares, Edito Model Lab
"""

#Import packages
import os
import s3fs
import matplotlib.pyplot as plt
import contextily as ctx
import numpy as np
import dfm_tools as dfmt
from matplotlib.colors import ListedColormap
import geopandas as gpd


In [ ]:
##USER DEFINED
# Input
nc_file =   'd_eco_impact_output.nc' #give the filename of the spatial data you want to plot (in case of partitions, use a '*' instead of the partition number to read all partitions at once)
directory = ''  #define if input data is saved in subfolder of s3 bucket

# Output location for plots
out= 'HSI_plots'

# Load validation data 
# example for the Zostera Noltei in the Wadden Sea is loaded automatically, otherwise upload your validation data.
shapefile_path = r'validation-data/zostera_noltei_2017Polygon_WGS84.shp' 

# Settings for mapfile
crs = "EPSG:4326"
layer=0 #as we consider 2D data 

In [ ]:
#Output directory
if not os.path.isdir(out):
   os.makedirs(out)

##Input
onxia_user_name = os.environ["GIT_USER_NAME"]
bucket_name = f"oidc-{onxia_user_name}"
S3_ENDPOINT_URL = os.environ["S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

# Download the nc-file
fs.download(f"{bucket_name}/{directory}/{nc_file}", nc_file)
print(f"{bucket_name}/{directory}/{nc_file}")

#Open map file
print('processing %s'%(os.path.basename(nc_file)))
uds_map = dfmt.open_partitioned_dataset(nc_file)
vars_pd = dfmt.get_ncvarproperties(uds_map)

#plot net/grid. use random variable and plot line to get grid
fig, ax = plt.subplots(figsize=(10,4))
pc = uds_map.grid.plot(edgecolor='crimson', linewidth=0.5)
if crs is None:
    ax.set_aspect('equal')
else:
    ctx.add_basemap(ax=ax, source=ctx.providers.Esri.WorldImagery, crs=crs, attribution=False)

# Create a custom colormap
cmap = plt.cm.Reds
cmap_with_transparency = cmap(np.arange(cmap.N))
cmap_with_transparency[0, -1] = 0  # Make the first color (corresponding to 0) fully transparent

# Create a new colormap from the modified array
transparent_cmap = ListedColormap(cmap_with_transparency)


In [ ]:
##Print available variables for manual check
uds_map

In [ ]:
# Plot variable- and HSI-maps in loop (choose from variable printed in previous blocks)

for variable in [
        'Temperature_2D_avg','HSI_Temperature',    
        'Salinity_2D_avg','HSI_Salinity',
        'Flow_Velocity_2D_max', 'HSI_Flow_Velocity',
        'Inundation_Time', 'HSI_Inundation_Time',
        'Total_HSI_MIN_month'
 ]:
    try:
        uds_map[variable]
        fig, ax = plt.subplots(figsize=(5,3))
        if vars_pd.loc[variable, 'shape'][0]>11:
            pc=uds_map[variable].isel(mesh2d_nLayers=layer,nmesh2d_layer=layer,missing_dims='ignore').mean(dim="time").ugrid.plot(cmap=transparent_cmap)
        else:
            pc=uds_map[variable].

        if crs is None:
            ax.set_aspect('equal')
        else:
            ctx.add_basemap(ax=ax, source=ctx.providers.Esri.WorldImagery, crs=crs, attribution=False)
        if variable == 'Total_HSI_MIN_month':
            ax.set_title(f"Simulated Habitat Suitability for ealgrass in the Dutch Wadden Sea in {np.unique(uds_map.time.dt.year)}", fontsize=12)
        plt.savefig(out + '/' + variable + '.png', bbox_inches='tight')
        print(variable)
    except:
        print(f'Variable {variable} is not available.')

In [ ]:
## Plot validation data
gdf = gpd.read_file(shapefile_path)
gdf_filtered = gdf[gdf['bedzostot'] > 0] # Filter non-zeroe values (specific to example validation dataset)

fig, ax = plt.subplots(figsize=(10,4))
plt.xlim(uds_map['mesh2d_face_x'].min(), uds_map['mesh2d_face_x'].max())
plt.ylim(uds_map['mesh2d_face_y'].min(), uds_map['mesh2d_face_y'].max())
gdf_filtered.plot(ax=ax, column='bedzostot', color='Red', edgecolor='Red')
if crs is None:
    ax.set_aspect('equal')
else:
    ctx.add_basemap(ax=ax, source=ctx.providers.Esri.WorldImagery, crs=crs, attribution=False)
plt.title(f"Observed seagreass (Zostera Noltii and littoral Zostera Marina) in {np.unique(uds_map.time.dt.year)}", fontsize=12) 
plt.savefig(out + '/validation.png', bbox_inches='tight')